### BikeSharing On Demand 
* 자전거 대여수를 예측 - 회귀
1. Data Loading
2. Data Preprocessing
3. Visualization
4. Feature Engineering
5. Model / Training / Prediction
7. Submission

### 1. Data Load

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('data/bike/train.csv', parse_dates=['datetime'])
test = pd.read_csv('data/bike/test.csv', parse_dates=['datetime'])
